# Javascript Dom私房手册

## Document

### 历遍dom

1. `parentNode`和`parentElement`有个很重要的知识点，就是`html`节点，即`document.documentElement`节点的`parentNode`是`document`，但是其`parentElement`为`null`，因为`document`并不是一个元素节点，`document`是一个`HTMLDocument`的实例。而所有的元素节点都是`Element`的实例。
```javascript
alert( document.documentElement.parentNode ); // document
alert( document.documentElement.parentElement ); // null
```
 在实际的使用中，当想从任意节点`elem`到`<html>`而不是到`document`时，这个细节可能很有用：
 
```javascript
while(elem = elem.parentElement) { // 向上，直到 <html>
  alert( elem );
}
```   

### 搜索：getElement*，querySelector*

1. `closest`有个知识点就是元素自身也会被搜索：
```html
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title></title>
  </head>
  <body>
    <h1>Contents</h1>
    <div class="contents">
      <ul class="book">
        <li class="chapter">Chapter 1</li>
        <li class="chapter">Chapter 1</li>
      </ul>
    </div>
    <script>
      let chapter = document.querySelector('.chapter'); // LI
      alert(chapter.closest('.book')); // UL
      alert(chapter.closest('.contents')); // DIV
      alert(chapter === chapter.closest('.chapter')); // true，因为自身也会被搜索
    </script>
  </body>
</html>
```
2. `getElements*`和`querySelectorAll`有个很重要的区别就是`getElements*`是实时的，即返回的集合会变化，始终反应文档的实际情况，而`querySelectorAll`是一个静态的集合。
3. 除了`matches`和`closest`，另一种用来检查子级与父级之间关系的方法就是利用元素节点的`contains`方法，不过要注意的是，如果是同一个元素的不同引用，比如`a`,`b`都引用同一个元素节点，那么`a.contains(b)`返回的也是`true`。
4. 对于有id特性的节点元素，会自动成为`window`对象的全局属性，但是注意，如果有连字符的特性，只能够通过方括号的方式进行访问。比如：
```javascript
let table = document.getElementById('age-table')
t = window['age-table']
console.log(t === table) // true
```

### 节点属性：type，tag 和 content

1. 可以通过节点的`tagName`或者`nodeName`查看节点的类型，`tagName`仅针对元素节点，返回的是标签的大写，文本节点没有`tagName`的属性，返回`undefined`，所有节点都有`nodeName`属性，元素节点返回标签大写，文本节点返回`#text`。
2. 如果`innerHTML`将一个`0<script>`标签插入到`document`中 — 它会成为HTML的一部分，但是不会执行。
3. `“innerHTML+=”`会进行完全重写，先删除旧的内容，然后插入新旧结合的内容，所有的图片和其他资源都将重写加载。
4. 与`innerHTML`不同，写入`outerHTML`不会改变元素。而是在`DOM`中替换它。比如：
```html
<div>Hello, world!</div>
<script>
  let div = document.querySelector('div');
  // 使用 <p>...</p> 替换 div.outerHTML
  div.outerHTML = '<p>A new element</p>'; // (*)
  // 蛤！'div' 还是原来那样！
  alert(div.outerHTML); // <div>Hello, world!</div> (**)
</script>
```
 - div 被从文档（document）中移除。
 - 另一个 HTML片段`<p>A new element</p>`被插入到其位置上。
 - div 仍拥有其旧的值。新的 HTML 没有被赋值给任何变量。
5. `textContent`允许以安全方式写入`html`，所有html标签均按字符串处理。

### 特性和属性

1. `dom`对象不包含非标准的html特性，但是可以通过`getAttribute`方法获取所有特性，不管是标准的还是非标准的。
2. `href`特性需要注意，其`elem.href`元素属性返回的总是一个绝对的URL，而不管html特性是如何设置的。
3. `data`特性是专门为程序员保留的接口，`data-*`可以通过dom的`dataset.*`属性进行访问。

### 修改文档

1. 所有插入方法都会自动从旧位置删除该节点。比如`document.body.append(elem)`，如果`elem`节点已经在`dom`树上了，则只会移动节点。

### 样式和类

1. 通过javascript重置样式，比如`display`属性，应该使用`elem.style.display=''`，此时会使用CSS样式或者浏览器默认样式。
2. `cssText`属性不是添加样式，而是整个替换，可以用`elem.setAttribute('style', 'color:red...')`来代替。
3. `getComputedStyle`返回解析后（注意不是计算后）的值，返回一个类似`style`的对象，注意此对象是只读的，从此对象返回属性，需要用完整的属性名，比如`getComputedStyle(elem).paddingTop`而不是`getComputedStyle(elem).padding`，后者可能会返回不正确的结果。

### 元素大小和滚动

1. 注意：`scrollLeft/scrollTop`是可写的，而且写入的话滚动条就会滚动。
2. 将`scrollTop`设置为0会使元素滚动到顶部/底部，书上说设置为`Infinity`会滚动到底部，其实只要设置为大于`scrollHeight-clientHeight`应该就会滚动到底部，因为文档滚动超出容器的部分最多也就是`scrollHeight-clientHeight`，就已经到底部，不会再向上滚动了。
3. CSS width与clientWidth的不同点，也是在javascript不使用`getComputedStyle`获取大小的原因：
 - clientWidth值是数值，而getComputedStyle(elem).width 返回一个以 px 作为后缀的字符串。
 - getComputedStyle可能会返回非数值的width，例如内联（inline）元素的 "auto"。
 - clientWidth是元素的内部内容区域加上padding，而CSS width（具有标准的 box-sizing）是内部内容区域，不包括 padding。
 - 如果有滚动条，并且浏览器为其保留了空间，那么某些浏览器的CSS width会减去该空间（因为它不再可用于内容），而有些不会。clientWidth属性总是相同的：如果有滚动条，则减去滚动条的大小。
4. `clientLeft/clientTop`可以获取到元素边框的大小。

### window大小和滚动

1. 文档可见部分的 width/height（内容区域的 width/height）：document.documentElement.clientWidth/Height
2. 整个文档的 width/height，其中包括滚动出去的部分：
```javascript
let scrollHeight = Math.max(
  document.body.scrollHeight, document.documentElement.scrollHeight,
  document.body.offsetHeight, document.documentElement.offsetHeight,
  document.body.clientHeight, document.documentElement.clientHeight
);
```
3. 读取当前的滚动：`window.pageYOffset/pageXOffset`。
4. `window.scrollBy(x,y)` — 相对当前位置进行滚动。
5. `elem.scrollIntoView(top)` — 滚动以使 elem 可见（elem 与窗口的顶部/底部对齐）。

### 坐标

1. 有两种坐标系：
 - 相对于窗口 — 类似于`position:fixed`，从窗口的顶部/左侧边缘计算得出。这些坐标表示为`clientX/clientY`。
 - 相对于文档 — 与文档根（document root）中的`position:absolute`类似，从文档的顶部/左侧边缘计算得出。表示为`pageX/pageY`。
2. 可以通过`getBoundingClientRect`方法获取元素坐标。返回一个`DOMRect`对象，有如下属性：
```javascript
{
  "x": 15,
  "y": -327.79998779296875,
  "width": 610.7999877929688,
  "height": 29.633331298828125,
  "top": -327.79998779296875,
  "right": 625.7999877929688,
  "bottom": -298.1666564941406,
  "left": 15
}
```
3. `getBoundingClientRect`获取窗口坐标，可以通过以下的辅助函数获取文档坐标：
```javascript
// 获取元素的文档坐标
function getCoords(elem) {
  let box = elem.getBoundingClientRect();
  return {
    top: box.top + window.pageYOffset,
    left: box.left + window.pageXOffset
  };
}
```

## 事件简介

### 浏览器事件简介

1. `addEventListener`可以接受一个对象作为参数，事件发生时，会调用这个对象的`handleEvent`方法。
2. 少数事件只能使用`addEventListener`方式，比如`transtionend`和`DOMContentLoaded`。

### 冒泡和捕获

1. `event.target`指发生事件的元素，`event.currentTarget`指当前正在运行处理程序的元素。
2. `event.stopPropagation()`停止向上移动，但是当前元素上的其他处理程序都会继续运行。`event.stopImmediatePropagation()`方法，可以用于停止冒泡，并阻止当前元素上的处理程序运行。使用该方法之后，其他处理程序就不会被执行。
3. 捕获事件使用`elem.addEventListener(..., {capture: true})`或者`elem.addEventListener(..., true)`

### 事件委托

1. 当将事件处理程序分配给`document`对象时，应该始终使用`addEventListener`，而不是`document.on<event>`，因为后者会引起冲突：新的处理程序会覆盖旧的处理程序。

### 浏览器默认行为

1. 主流的方式是使用 event 对象。有一个 event.preventDefault() 方法。如果处理程序是使用`on<event>`（而不是`addEventListener`）分配的，那返回 `false`也行。
2. `addEventListener`的可选项`passive: true`向浏览器发出信号，表明处理程序将不会调用`preventDefault()`。

## 各种库的使用

### js-xlsx

- [如何使用JavaScript实现纯前端读取和导出excel文件](https://www.cnblogs.com/liuxianan/p/js-excel.html#%E5%88%A9%E7%94%A8%E5%AE%98%E6%96%B9%E5%B7%A5%E5%85%B7%E7%B1%BB%E7%94%9F%E6%88%90)